# 어뷰징 탐지 모델 개선

기존 모델의 과적합 문제를 해결하고, 더 robust한 모델을 개발합니다.

## 개선 항목
1. K-Fold 교차검증으로 모델 안정성 검증
2. **Feature Selection으로 불필요한 피처 제거**
3. 하이퍼파라미터 튜닝 (GridSearchCV)
4. **Early Stopping으로 과적합 방지**
5. 앙상블 기법 (Voting, Stacking)
6. 클래스 불균형 처리
7. **Learning Curve로 과적합 진단**
8. SHAP 분석으로 모델 해석성 개선

In [ ]:
import os
import warnings

import joblib
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.ensemble import (
    AdaBoostClassifier,
    GradientBoostingClassifier,
    RandomForestClassifier,
    StackingClassifier,
    VotingClassifier,
)
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    precision_score,
    recall_score,
    roc_auc_score,
    roc_curve,
)
from sklearn.model_selection import (
    GridSearchCV,
    StratifiedKFold,
    cross_val_score,
    train_test_split,
    learning_curve,  # 과적합 진단용
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.feature_selection import SelectFromModel, RFE  # Feature Selection
from src.database import load_table
from src.features.feature_generation import FeatureGenerator

warnings.filterwarnings('ignore')

print("라이브러리 로드 완료")

## 1. 데이터 로드 및 전처리

In [ ]:
sellers_df = load_table('sellers')
products_df = load_table('products')
reviews_df = load_table('reviews')
questions_df = load_table('questions')

print(f"판매자: {len(sellers_df)}개")
print(f"상품: {len(products_df)}개")
print(f"리뷰: {len(reviews_df)}개")
print(f"질문: {len(questions_df)}개")

In [ ]:
# 피처 엔지니어링
generator = FeatureGenerator(
    sellers_df=sellers_df,
    products_df=products_df,
    reviews_df=reviews_df,
    questions_df=questions_df
)
features_df = generator.generate_legacy_features()

print(f"피처 데이터: {features_df.shape}")

In [ ]:
# 피처와 타겟 분리
feature_columns = [
    'satisfaction_score', 'review_count', 'total_product_count',
    'product_count_actual', 'price_mean', 'price_std', 'price_min', 'price_max',
    'rating_mean', 'rating_std', 'review_sum', 'review_mean',
    'discount_mean', 'discount_max', 'shipping_fee_mean', 'shipping_days_mean',
    'review_count_actual', 'review_rating_mean', 'review_rating_std',
    'review_length_mean', 'review_length_std', 'review_length_max',
    'question_count', 'answer_rate'
]

X = features_df[feature_columns]
y = features_df['is_abusing_seller'].astype(int)

# Train/Test 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"훈련 세트: {X_train.shape[0]}개 (어뷰징: {y_train.sum()}개, {y_train.mean()*100:.1f}%)")
print(f"테스트 세트: {X_test.shape[0]}개 (어뷰징: {y_test.sum()}개, {y_test.mean()*100:.1f}%)")

## 2. K-Fold 교차검증으로 과적합 검증

기존 Random Forest 100% 정확도가 과적합인지 확인합니다.

In [ ]:
# Stratified K-Fold 교차검증
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

models_cv = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=42),
    'SVM': SVC(probability=True, random_state=42),
    'KNN': KNeighborsClassifier(n_neighbors=5),
    'AdaBoost': AdaBoostClassifier(n_estimators=100, random_state=42)
}

cv_results = []

print("5-Fold 교차검증 수행 중...\n")
for name, model in models_cv.items():
    # 스케일링이 필요한 모델
    if name in ['Logistic Regression', 'SVM', 'KNN']:
        X_cv = X_train_scaled
    else:
        X_cv = X_train
    
    # 여러 메트릭으로 교차검증
    acc_scores = cross_val_score(model, X_cv, y_train, cv=cv, scoring='accuracy')
    f1_scores = cross_val_score(model, X_cv, y_train, cv=cv, scoring='f1')
    roc_scores = cross_val_score(model, X_cv, y_train, cv=cv, scoring='roc_auc')
    
    cv_results.append({
        'model': name,
        'acc_mean': acc_scores.mean(),
        'acc_std': acc_scores.std(),
        'f1_mean': f1_scores.mean(),
        'f1_std': f1_scores.std(),
        'roc_mean': roc_scores.mean(),
        'roc_std': roc_scores.std()
    })
    
    print(f"{name}:")
    print(f"  Accuracy: {acc_scores.mean():.4f} (+/- {acc_scores.std():.4f})")
    print(f"  F1-Score: {f1_scores.mean():.4f} (+/- {f1_scores.std():.4f})")
    print(f"  ROC-AUC:  {roc_scores.mean():.4f} (+/- {roc_scores.std():.4f})")
    print()

In [ ]:
# 교차검증 결과 시각화
cv_df = pd.DataFrame(cv_results)

fig = make_subplots(rows=1, cols=3, subplot_titles=('Accuracy', 'F1-Score', 'ROC-AUC'))

colors = px.colors.qualitative.Set2

for i, (metric, title) in enumerate([('acc', 'Accuracy'), ('f1', 'F1-Score'), ('roc', 'ROC-AUC')], 1):
    fig.add_trace(
        go.Bar(
            x=cv_df['model'],
            y=cv_df[f'{metric}_mean'],
            error_y=dict(type='data', array=cv_df[f'{metric}_std']),
            marker_color=colors[:len(cv_df)],
            showlegend=False
        ),
        row=1, col=i
    )

fig.update_layout(
    title='5-Fold 교차검증 결과 (평균 ± 표준편차)',
    height=400,
    template='plotly_white'
)
fig.update_xaxes(tickangle=45)
fig.show()

## 2.1 Feature Selection (과적합 방지)

불필요한 피처를 제거하여 모델의 일반화 성능을 높입니다.
- 피처가 많으면 노이즈에 과적합될 위험이 증가
- 중요 피처만 선택하여 모델 복잡도 감소

In [ ]:
# Feature Selection: Random Forest 기반 중요도로 피처 선택
print("=== Feature Selection (과적합 방지) ===\n")

# 1. 초기 RF로 피처 중요도 계산
rf_selector = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=10)
rf_selector.fit(X_train, y_train)

# 피처 중요도 시각화
feature_importance = pd.DataFrame({
    'feature': feature_columns,
    'importance': rf_selector.feature_importances_
}).sort_values('importance', ascending=True)

fig = px.bar(feature_importance, x='importance', y='feature', orientation='h',
             title='피처 중요도 (Random Forest)', height=600)
fig.show()

# 2. SelectFromModel로 중요 피처 선택 (평균 이상 중요도)
selector = SelectFromModel(rf_selector, threshold='mean', prefit=True)
X_train_selected = selector.transform(X_train)
X_test_selected = selector.transform(X_test)

selected_features = [f for f, s in zip(feature_columns, selector.get_support()) if s]
print(f"선택된 피처 ({len(selected_features)}개): {selected_features}")
print(f"제거된 피처 ({len(feature_columns) - len(selected_features)}개)")

# 3. 선택된 피처로 CV 성능 비교
rf_full = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=10)
rf_selected = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=10)

cv_full = cross_val_score(rf_full, X_train, y_train, cv=cv, scoring='f1')
cv_selected = cross_val_score(rf_selected, X_train_selected, y_train, cv=cv, scoring='f1')

print(f"\n전체 피처 CV F1: {cv_full.mean():.4f} (+/- {cv_full.std():.4f})")
print(f"선택 피처 CV F1: {cv_selected.mean():.4f} (+/- {cv_selected.std():.4f})")

# 성능이 크게 떨어지지 않으면 선택된 피처 사용
USE_SELECTED_FEATURES = cv_selected.mean() >= cv_full.mean() - 0.02
print(f"\n→ 선택된 피처 사용: {USE_SELECTED_FEATURES}")

## 3. 하이퍼파라미터 튜닝

In [ ]:
# Random Forest 하이퍼파라미터 튜닝 (과적합 방지 강화)
print("Random Forest 하이퍼파라미터 튜닝 중...")

rf_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7, 10],           # 더 얕은 트리 추가 (과적합 방지)
    'min_samples_split': [5, 10, 20],     # 더 큰 값 추가 (과적합 방지)
    'min_samples_leaf': [2, 4, 8],        # 더 큰 값 추가 (과적합 방지)
    'max_features': ['sqrt', 'log2', 0.5], # 피처 샘플링 (dropout 효과)
    'class_weight': ['balanced', None]
}

rf_grid = GridSearchCV(
    RandomForestClassifier(random_state=42, oob_score=True),  # OOB score로 일반화 성능 확인
    rf_param_grid,
    cv=cv,
    scoring='f1',
    n_jobs=-1,
    verbose=1
)

rf_grid.fit(X_train, y_train)

print(f"\n최적 파라미터: {rf_grid.best_params_}")
print(f"최고 CV F1-Score: {rf_grid.best_score_:.4f}")

# OOB Score 확인 (과적합 여부 판단)
best_rf_temp = rf_grid.best_estimator_
if hasattr(best_rf_temp, 'oob_score_'):
    print(f"OOB Score: {best_rf_temp.oob_score_:.4f}")
    gap = rf_grid.best_score_ - best_rf_temp.oob_score_
    print(f"CV - OOB Gap: {gap:.4f} {'(과적합 의심)' if gap > 0.05 else '(양호)'}")

In [ ]:
# Gradient Boosting 하이퍼파라미터 튜닝 (Early Stopping 포함)
print("Gradient Boosting 하이퍼파라미터 튜닝 중...")

# Early Stopping을 위한 Validation Set 분리
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42, stratify=y_train
)

gb_param_grid = {
    'n_estimators': [100, 200, 300],       # 충분히 크게 (Early Stopping이 조절)
    'learning_rate': [0.01, 0.05, 0.1],    # 작은 학습률 추가
    'max_depth': [2, 3, 4, 5],             # 더 얕은 트리 추가
    'min_samples_split': [5, 10, 20],
    'min_samples_leaf': [2, 4, 8],
    'subsample': [0.7, 0.8, 0.9],          # 더 낮은 샘플링 추가
    'max_features': ['sqrt', 0.5]          # 피처 샘플링 추가
}

gb_grid = GridSearchCV(
    GradientBoostingClassifier(
        random_state=42,
        validation_fraction=0.15,          # Early Stopping용 validation
        n_iter_no_change=10,               # 10회 개선 없으면 중단
        tol=1e-4
    ),
    gb_param_grid,
    cv=cv,
    scoring='f1',
    n_jobs=-1,
    verbose=1
)

gb_grid.fit(X_train, y_train)

print(f"\n최적 파라미터: {gb_grid.best_params_}")
print(f"최고 CV F1-Score: {gb_grid.best_score_:.4f}")

# 실제 사용된 트리 수 확인 (Early Stopping 효과)
best_gb_temp = gb_grid.best_estimator_
print(f"실제 사용 트리 수: {best_gb_temp.n_estimators_} / {best_gb_temp.n_estimators} (Early Stop 효과)")

In [ ]:
# Logistic Regression 하이퍼파라미터 튜닝 (ElasticNet 추가)
print("Logistic Regression 하이퍼파라미터 튜닝 중...")

lr_param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10],        # 더 강한 정규화 포함
    'penalty': ['l1', 'l2', 'elasticnet'], # ElasticNet 추가
    'solver': ['saga'],                     # ElasticNet 지원 solver
    'l1_ratio': [0.3, 0.5, 0.7],           # ElasticNet 비율
    'class_weight': ['balanced', None]
}

lr_grid = GridSearchCV(
    LogisticRegression(random_state=42, max_iter=2000),
    lr_param_grid,
    cv=cv,
    scoring='f1',
    n_jobs=-1,
    verbose=1
)

lr_grid.fit(X_train_scaled, y_train)

print(f"\n최적 파라미터: {lr_grid.best_params_}")
print(f"최고 CV F1-Score: {lr_grid.best_score_:.4f}")

# 계수 분석 (0에 가까운 계수 = 중요하지 않은 피처)
best_lr_temp = lr_grid.best_estimator_
coef_df = pd.DataFrame({
    'feature': feature_columns,
    'coefficient': np.abs(best_lr_temp.coef_[0])
}).sort_values('coefficient', ascending=False)
print(f"\n정규화로 제거된 피처 수: {(coef_df['coefficient'] < 0.01).sum()}개")

## 4. 앙상블 모델 구축

In [ ]:
# 튜닝된 모델들로 앙상블 구성
best_rf = rf_grid.best_estimator_
best_gb = gb_grid.best_estimator_
best_lr = lr_grid.best_estimator_

# Voting Classifier (Soft Voting)
voting_clf = VotingClassifier(
    estimators=[
        ('rf', best_rf),
        ('gb', best_gb),
        ('lr', LogisticRegression(**lr_grid.best_params_, random_state=42, max_iter=1000))
    ],
    voting='soft'
)

# 스케일링된 데이터로 학습 (LR 때문에)
# 주의: 실제로는 파이프라인을 사용해야 하지만, 여기서는 RF/GB가 스케일링에 민감하지 않아 간소화
voting_clf.fit(X_train_scaled, y_train)

# 교차검증
voting_cv_scores = cross_val_score(voting_clf, X_train_scaled, y_train, cv=cv, scoring='f1')
print(f"Voting Classifier CV F1: {voting_cv_scores.mean():.4f} (+/- {voting_cv_scores.std():.4f})")

In [ ]:
# Stacking Classifier
stacking_clf = StackingClassifier(
    estimators=[
        ('rf', RandomForestClassifier(**rf_grid.best_params_, random_state=42)),
        ('gb', GradientBoostingClassifier(**gb_grid.best_params_, random_state=42)),
        ('lr', LogisticRegression(**lr_grid.best_params_, random_state=42, max_iter=1000))
    ],
    final_estimator=LogisticRegression(random_state=42, max_iter=1000),
    cv=5
)

stacking_clf.fit(X_train_scaled, y_train)

# 교차검증
stacking_cv_scores = cross_val_score(stacking_clf, X_train_scaled, y_train, cv=cv, scoring='f1')
print(f"Stacking Classifier CV F1: {stacking_cv_scores.mean():.4f} (+/- {stacking_cv_scores.std():.4f})")

## 4.1 Learning Curve 분석 (과적합 진단)

Learning Curve는 훈련 데이터 크기에 따른 성능 변화를 보여줍니다:
- **Train Score >> Validation Score**: 과적합 (모델이 너무 복잡)
- **Train Score ≈ Validation Score (둘 다 낮음)**: 과소적합 (모델이 너무 단순)
- **Train Score ≈ Validation Score (둘 다 높음)**: 이상적인 상태

In [ ]:
# Learning Curve 계산 및 시각화
print("Learning Curve 분석 중...")

# 튜닝된 RF 모델로 Learning Curve 계산
train_sizes, train_scores, val_scores = learning_curve(
    best_rf, X_train, y_train,
    train_sizes=np.linspace(0.1, 1.0, 10),
    cv=cv,
    scoring='f1',
    n_jobs=-1
)

# 평균 및 표준편차 계산
train_mean = train_scores.mean(axis=1)
train_std = train_scores.std(axis=1)
val_mean = val_scores.mean(axis=1)
val_std = val_scores.std(axis=1)

# Plotly로 시각화
fig = go.Figure()

# Training Score
fig.add_trace(go.Scatter(
    x=train_sizes, y=train_mean,
    mode='lines+markers',
    name='Training Score',
    line=dict(color='blue'),
    error_y=dict(type='data', array=train_std, visible=True)
))

# Validation Score
fig.add_trace(go.Scatter(
    x=train_sizes, y=val_mean,
    mode='lines+markers',
    name='Validation Score',
    line=dict(color='green'),
    error_y=dict(type='data', array=val_std, visible=True)
))

fig.update_layout(
    title='Learning Curve (Tuned Random Forest)',
    xaxis_title='Training Set Size',
    yaxis_title='F1 Score',
    template='plotly_white',
    height=500
)
fig.show()

# 과적합 진단
gap = train_mean[-1] - val_mean[-1]
print(f"\n=== 과적합 진단 ===")
print(f"최종 Training F1: {train_mean[-1]:.4f}")
print(f"최종 Validation F1: {val_mean[-1]:.4f}")
print(f"Train-Val Gap: {gap:.4f}")

if gap > 0.1:
    print("⚠️ 과적합 가능성 높음 - 모델 복잡도를 줄이거나 정규화 강화 필요")
elif gap > 0.05:
    print("⚡ 약간의 과적합 - 주의 필요")
else:
    print("✅ 과적합 없음 - 모델이 잘 일반화됨")

## 5. 최종 모델 평가

In [ ]:
# 모든 모델 테스트 세트 평가
final_models = {
    'Tuned RF': (best_rf, X_test),
    'Tuned GB': (best_gb, X_test),
    'Tuned LR': (best_lr, X_test_scaled),
    'Voting': (voting_clf, X_test_scaled),
    'Stacking': (stacking_clf, X_test_scaled)
}

final_results = []

for name, (model, X_eval) in final_models.items():
    y_pred = model.predict(X_eval)
    y_proba = model.predict_proba(X_eval)[:, 1]
    
    results = {
        'model': name,
        'accuracy': accuracy_score(y_test, y_pred),
        'precision': precision_score(y_test, y_pred),
        'recall': recall_score(y_test, y_pred),
        'f1': f1_score(y_test, y_pred),
        'roc_auc': roc_auc_score(y_test, y_proba)
    }
    final_results.append(results)

final_df = pd.DataFrame(final_results)
print("=== 최종 모델 성능 비교 (테스트 세트) ===")
print(final_df.to_string(index=False))

In [ ]:
# 성능 비교 시각화
metrics = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']
metric_names = ['정확도', '정밀도', '재현율', 'F1-Score', 'ROC-AUC']

fig = go.Figure()

for _, row in final_df.iterrows():
    fig.add_trace(go.Bar(
        name=row['model'],
        x=metric_names,
        y=[row[m] for m in metrics]
    ))

fig.update_layout(
    title='튜닝된 모델 성능 비교 (테스트 세트)',
    barmode='group',
    yaxis_title='Score',
    template='plotly_white',
    height=500
)
fig.show()

In [ ]:
# ROC Curve 비교
fig = go.Figure()

for name, (model, X_eval) in final_models.items():
    y_proba = model.predict_proba(X_eval)[:, 1]
    fpr, tpr, _ = roc_curve(y_test, y_proba)
    auc = roc_auc_score(y_test, y_proba)
    
    fig.add_trace(go.Scatter(
        x=fpr, y=tpr,
        name=f'{name} (AUC={auc:.3f})',
        mode='lines'
    ))

fig.add_trace(go.Scatter(
    x=[0, 1], y=[0, 1],
    name='Random',
    mode='lines',
    line=dict(dash='dash', color='gray')
))

fig.update_layout(
    title='튜닝된 모델 ROC Curve 비교',
    xaxis_title='False Positive Rate',
    yaxis_title='True Positive Rate',
    template='plotly_white'
)
fig.show()

## 6. SHAP 분석 (모델 해석성)

In [ ]:
# SHAP 설치 확인
try:
    import shap
    print(f"SHAP 버전: {shap.__version__}")
except ImportError:
    print("SHAP 설치 중...")
    !uv add shap
    import shap
    print("SHAP 설치 완료")

In [ ]:
# 최적 모델 선택 (F1 기준)
best_model_name = final_df.loc[final_df['f1'].idxmax(), 'model']
print(f"SHAP 분석 대상 모델: {best_model_name}")

# TreeExplainer 사용 (RF, GB)
if 'RF' in best_model_name:
    explainer = shap.TreeExplainer(best_rf)
    shap_values = explainer.shap_values(X_test)
    if isinstance(shap_values, list):
        shap_values = shap_values[1]  # 어뷰징 클래스
elif 'GB' in best_model_name:
    explainer = shap.TreeExplainer(best_gb)
    shap_values = explainer.shap_values(X_test)
else:
    # 다른 모델의 경우 KernelExplainer 사용
    model, X_eval = final_models[best_model_name]
    explainer = shap.KernelExplainer(model.predict_proba, shap.sample(X_train_scaled, 100))
    shap_values = explainer.shap_values(X_eval[:50])

In [ ]:
# SHAP Beeswarm Plot (피처 영향 방향) - Plotly 구현

# shap_values가 3차원(samples, features, classes)이거나 리스트인 경우 처리
if isinstance(shap_values, list):
    shap_values_2d = shap_values[1]  # Positive class
elif hasattr(shap_values, 'shape') and len(shap_values.shape) == 3:
    shap_values_2d = shap_values[:, :, 1]  # Positive class
else:
    shap_values_2d = shap_values

# 데이터 준비
shap_df = pd.DataFrame(shap_values_2d, columns=feature_columns)

# 색상을 위해 스케일링된 데이터 사용 (각 피처별로 Low/High 구분 명확화)
feature_df = pd.DataFrame(X_test_scaled, columns=feature_columns)

# 시각화를 위해 데이터 변환 (Long Format)
shap_melted = shap_df.melt(var_name='feature', value_name='shap_value')
feature_melted = feature_df.melt(var_name='feature', value_name='feature_value')

plot_df = shap_melted.copy()
plot_df['feature_value'] = feature_melted['feature_value']

# 중요도 순으로 정렬
mean_abs_shap = np.abs(shap_values_2d).mean(axis=0)
importance_df = pd.DataFrame({
    'feature': feature_columns,
    'importance': mean_abs_shap
}).sort_values('importance', ascending=True)
sorted_features = importance_df['feature'].tolist()

# Plotly Strip Plot
fig = px.strip(
    plot_df, 
    x='shap_value', 
    y='feature', 
    color='feature_value',
    category_orders={'feature': sorted_features},
    title='SHAP 피처 영향 분석 (Beeswarm Style)',
    height=800
)

fig.update_layout(
    xaxis_title='SHAP Value (impact on model output)',
    yaxis_title='Feature',
    coloraxis_colorscale='RdBu_r'
)
fig.show()

In [ ]:
# SHAP Summary Plot (피처 중요도) - Plotly 구현
# 위에서 계산한 importance_df 사용

fig = px.bar(
    importance_df, 
    x='importance', 
    y='feature', 
    orientation='h',
    title='SHAP 피처 중요도 (Mean Absolute SHAP Value)',
    height=600
)
fig.update_layout(
    xaxis_title='mean(|SHAP value|)',
    yaxis_title='Feature'
)
fig.show()

In [ ]:
# SHAP Waterfall 시각화 (src/visualize에서 함수 import)
from src.visualize import plot_shap_waterfall

# 첫 번째 어뷰징 샘플 시각화
abusing_idx = y_test[y_test == 1].index[0]
sample_idx = list(y_test.index).index(abusing_idx)

# SHAP 값 추출 (Class 1에 대한 값)
sv_class1 = shap_values[sample_idx][:, 1]
base_value = explainer.expected_value[1]
features = X_test.iloc[sample_idx]

# Waterfall 차트 그리기
fig = plot_shap_waterfall(
    shap_values=sv_class1,
    sample_idx=sample_idx,
    feature_columns=feature_columns,
    feature_values=features,
    base_value=base_value,
    top_n=20
)
fig.show()

## 7. 최종 모델 저장

In [ ]:
# 최고 성능 모델 저장
best_idx = final_df['f1'].idxmax()
best_model_name = final_df.loc[best_idx, 'model']
best_model_obj = final_models[best_model_name][0]

os.makedirs('../models', exist_ok=True)

# 모델 저장
model_filename = f'abusing_detector_tuned_{best_model_name.lower().replace(" ", "_")}.pkl'
joblib.dump(best_model_obj, f'../models/{model_filename}')
joblib.dump(scaler, '../models/scaler_tuned.pkl')

# 튜닝 파라미터 저장
tuning_results = {
    'rf_params': rf_grid.best_params_,
    'gb_params': gb_grid.best_params_,
    'lr_params': lr_grid.best_params_,
    'best_model': best_model_name,
    'best_f1': final_df.loc[best_idx, 'f1'],
    'cv_results': cv_results
}
joblib.dump(tuning_results, '../models/tuning_results.pkl')

print(f"최종 모델 저장 완료: models/{model_filename}")
print(f"스케일러 저장 완료: models/scaler_tuned.pkl")
print(f"튜닝 결과 저장 완료: models/tuning_results.pkl")

## 8. 요약

In [ ]:
print("="*70)
print("어뷰징 탐지 모델 개선 완료")
print("="*70)

print("\n[1] 과적합 방지 기법 적용")
print("-" * 50)
print("✓ Feature Selection: 중요 피처만 선택 (SelectFromModel)")
print("✓ max_features: 피처 샘플링으로 dropout 효과")
print("✓ Early Stopping: GB에서 validation loss 기반 조기 종료")
print("✓ 강한 정규화: max_depth 제한, min_samples 증가")
print("✓ ElasticNet: L1+L2 혼합 정규화")
print("✓ Learning Curve: Train-Val gap으로 과적합 진단")
print("✓ OOB Score: RF에서 out-of-bag 샘플로 일반화 성능 확인")

print("\n[2] 교차검증 결과")
print("-" * 50)
cv_summary = pd.DataFrame(cv_results)[['model', 'f1_mean', 'f1_std']].sort_values('f1_mean', ascending=False)
print(cv_summary.to_string(index=False))

print("\n[3] 하이퍼파라미터 튜닝 결과")
print("-" * 50)
print(f"Random Forest 최적 파라미터:")
for k, v in rf_grid.best_params_.items():
    print(f"  - {k}: {v}")
print(f"\nGradient Boosting 최적 파라미터:")
for k, v in gb_grid.best_params_.items():
    print(f"  - {k}: {v}")

print("\n[4] 최종 모델 성능 (테스트 세트)")
print("-" * 50)
print(final_df.to_string(index=False))

print(f"\n[5] 최종 선택 모델: {best_model_name}")
print(f"    F1-Score: {final_df.loc[best_idx, 'f1']:.4f}")
print(f"    ROC-AUC: {final_df.loc[best_idx, 'roc_auc']:.4f}")

# Learning Curve 결과 요약
print(f"\n[6] 과적합 진단 결과")
print("-" * 50)
print(f"    Train-Val Gap: {gap:.4f}")
if gap > 0.1:
    print("    상태: ⚠️ 과적합 가능성 높음")
elif gap > 0.05:
    print("    상태: ⚡ 약간의 과적합")
else:
    print("    상태: ✅ 양호 (과적합 없음)")